# Exploration des Données DVF - Île-de-France

Ce notebook permet d'explorer les données immobilières DVF pour l'Île-de-France.

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from src.data.dvf_downloader import DVFDownloader
from src.data.data_cleaner import DataCleaner
from src.analysis.price_analyzer import PriceAnalyzer

# Configuration visualisation
sns.set_theme(style='whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

%load_ext autoreload
%autoreload 2

## 1. Téléchargement des Données

Téléchargement des données DVF pour l'année spécifiée.

In [ ]:
# Configuration
YEAR = 2023

# Téléchargement
downloader = DVFDownloader()
files = downloader.download_idf_data(year=YEAR)

## 2. Chargement et Nettoyage

In [ ]:
# Charger les données brutes
df_raw = downloader.load_idf_data(year=YEAR)
print(f"Données brutes: {len(df_raw):,} lignes")
df_raw.head()

In [ ]:
# Informations sur les colonnes
df_raw.info()

In [ ]:
# Nettoyage des données
cleaner = DataCleaner()
df_clean = cleaner.clean_dvf_data(df_raw)

print(f"\nDonnées nettoyées: {len(df_clean):,} lignes")
df_clean.head()

In [ ]:
# Sauvegarder les données nettoyées
cleaner.save_cleaned_data(df_clean, year=YEAR)

## 3. Analyse des Prix

In [ ]:
# Initialiser l'analyzer
analyzer = PriceAnalyzer(df=df_clean)

In [ ]:
# Statistiques pour Paris
paris_stats = analyzer.get_city_stats("Paris")
if paris_stats:
    print(f"Paris - Statistiques {YEAR}")
    print(f"  Prix moyen: {paris_stats.prix_moyen_m2:,.0f} €/m²")
    print(f"  Prix médian: {paris_stats.prix_median_m2:,.0f} €/m²")
    print(f"  Prix min: {paris_stats.prix_min_m2:,.0f} €/m²")
    print(f"  Prix max: {paris_stats.prix_max_m2:,.0f} €/m²")
    print(f"  Transactions: {paris_stats.nombre_transactions:,}")

In [ ]:
# Analyser toutes les villes
all_stats = analyzer.analyze_all_cities()
print(f"Total: {len(all_stats)} villes analysées")
all_stats.head(10)

## 4. Visualisations

In [ ]:
# Top 15 villes les plus chères
top_15 = all_stats.head(15)

plt.figure(figsize=(12, 8))
plt.barh(top_15['ville'], top_15['prix_moyen_m2'], color='steelblue')
plt.xlabel('Prix moyen (€/m²)', fontsize=12)
plt.title(f'Top 15 des Villes les Plus Chères en Île-de-France ({YEAR})', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Distribution des prix par département
plt.figure(figsize=(14, 6))
sns.boxplot(data=df_clean, x='code_departement', y='prix_m2', palette='Set2')
plt.xlabel('Département', fontsize=12)
plt.ylabel('Prix au m² (€)', fontsize=12)
plt.title(f'Distribution des Prix au m² par Département ({YEAR})', fontsize=14, fontweight='bold')
plt.ylim(0, 20000)
plt.tight_layout()
plt.show()

In [ ]:
# Nombre de transactions par département
transactions_dept = df_clean.groupby('code_departement').size().sort_values(ascending=False)

plt.figure(figsize=(10, 6))
transactions_dept.plot(kind='bar', color='coral')
plt.xlabel('Département', fontsize=12)
plt.ylabel('Nombre de transactions', fontsize=12)
plt.title(f'Nombre de Transactions par Département ({YEAR})', fontsize=14, fontweight='bold')
plt.xticks(rotation=0)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Relation surface vs prix
sample = df_clean.sample(min(5000, len(df_clean)))  # Échantillon pour la lisibilité

plt.figure(figsize=(10, 6))
plt.scatter(sample['surface_reelle_bati'], sample['prix_m2'], alpha=0.3, s=10)
plt.xlabel('Surface (m²)', fontsize=12)
plt.ylabel('Prix au m² (€)', fontsize=12)
plt.title(f'Relation Surface vs Prix au m² ({YEAR})', fontsize=14, fontweight='bold')
plt.xlim(0, 200)
plt.ylim(0, 20000)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## 5. Export des Résultats

In [ ]:
# Exporter l'analyse complète
analyzer.export_analysis(all_stats, filename=f'analyse_idf_{YEAR}.xlsx')
print(f"✓ Résultats exportés")